# Pima Indians Diabetes Database
    outcome is only 0 nor 1

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
data = pd.read_csv("./data/pima-data.csv")
data.isnull().values.any()
data.head(7)
data.shape

In [ ]:
corrmat = data.corr()
top_corr_features = corrmat.index
plt.figure(figsize=(20, 20))

# ploting correlation
g = sns.heatmap(data[top_corr_features].corr(), 
                annot=True, cmap="RdYlGn")

In [ ]:
# Making outcome data form bool type to 0/1

diabetes_map = {True: 1, False: 0}
data['diabetes'] = data['diabetes'].map(diabetes_map)
data.head(7)

In [ ]:
diabetes_true_count = len(data.loc[data['diabetes']== True])
diabetes_false_count = len(data.loc[data['diabetes']== False])
print(f"nb_body_attein: {diabetes_true_count}\n nb_body_sane: {diabetes_false_count}")

In [ ]:
from sklearn.model_selection import train_test_split

feature_columns = ['num_preg', 'glucose_conc', 'insulin', 'bmi', 'diab_pred', 'age', 'skin']
predict_class = ['diabetes']

In [ ]:
X = data[feature_columns].values
y= data[predict_class].values

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=10)

Check how many other missing values

In [ ]:
len(data.loc[data['age']==0])
len(data.loc[data['skin']==0])
len(data.loc[data['insuline']==0])

In [ ]:
from sklearn.preprocessing import Inputer
from sklearn.ensemble import RandomForestClassifier


fill_values = Inputer(missing_values=0, strategy="mean", axis=0)
X_train = fill_values.fit_transform(X_train)
X_test = fill_values.fit_transform(X_test)

In [ ]:
random_forest_model = RandomForestClassifier(random_state=0)
random_forest_model.fit(X_train, y_train.ravel())

In [ ]:
predict_train_data = random_forest_model.predict(X_test)

from sklearn import metrics
print("Acuracy: ", metrics.accuracy_score(y_test, predict_train_data))

# Hyperparameter Optimization

In [ ]:
params = {
    "learning_rate":    [0.05, 0.10, 0.15, 0.20, 0.25, 0.30],
    "max_depth":        [3, 4, 5, 6, 8, 10, 12, 15],
    "min_child_wieght": [1, 3, 5, 7],
    "gama":             [0.0, 0.1, 0.2, 0.3, 0.4],
    "colsample":        [0.3, 0.4, 0.5, 0.7]
}

In [ ]:
import xgboost, datetime
from sklearn.model_selection import RandomizedSearchCV


classifier = xgboost.XGBClassifier()
random_search = RandomForestClassifier(classifier, **kwargs=params, n_iter=5, scoring='roc_auc', n_jobs=-1, cv=5, verbose=3)

In [ ]:
def timer(start_time=None):
    if not start_time:
        start_time=datetime.now
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now()-start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print(f'\n Time taken: {thour}-{tmin}-{round(tsec, 2)}')

In [ ]:
from datetime import datetime

start_time = timer(None)
random_search.fit(X_train, y_train.ravel())
timer(start_time)

In [ ]:
classifier = random_search.base_estimator_
classifier

In [ ]:
from sklearn.model_selection import cross_val_score

score = cross_val_score(classifier, X_train, y_train.ravel(), cv=10)
print(score)
score.mean()